<p align="center">
    <a href="https://predict-idlab.github.io/landmarker">
        <img alt="landmarker" src="https://raw.githubusercontent.com/predict-idlab/landmarker/main/docs/_static/images/logo.svg" width="66%">
    </a>
</p>

# Training and Evaluating Adaptive Heatmap Regression Model for Landmark Detection with ISBI 2015 Cephalometric X-ray Dataset

In this tutorial, we will train and evaluate an adaptive heatmap regression model for landmark 
detection with the ISBI 2015 Cephalometric X-ray dataset. The ISBI 2015 Cephalometric X-ray dataset
is a dataset of 2D cephalometric X-rays. The dataset contains 400 images, each with 19 landmarks
annotated. The dataset is split into a training set of 150 images and two test sets of 150 and
100 images respectively.

We will go through the following steps:
* [Loading the dataset](#Loading-the-dataset)
* [Constructing a heatmap generator](#Constructing-a-heatmap-generator)
* [Inspecting the dataset](#Inspecting-the-dataset)
* [Training and initializing the SpatialConfiguration model](#Training-the-model)
* [Evaluating the model](#Evaluating-the-model)

<a target="_blank" href="https://colab.research.google.com/github/predict-idlab/landmarker/examples/adaptive_scn_isbi2015.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup environment

In [ ]:
%load_ext autoreload
%autoreload 2


# !python -c "import landmarker" || pip install landmarker

import sys
import os
import ra_utils.visualization.plot_landmarks #.plot_landmarks

# sys.path.append("../src/")
import landmarker
from landmarker.datasets import get_cepha_landmark_datasets, get_tiny_cepha_landmark_datasets


import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from monai.transforms import (Compose, RandAffined, RandGaussianNoised, RandStdShiftIntensityd,
                              RandScaleIntensityd, RandAdjustContrastd, RandHistogramShiftd,
                              ScaleIntensityd, Lambdad, LoadImage, Transpose)
from tqdm.notebook import tqdm


import pandas as pd
from pathlib import Path
import ra_utils
import ra_utils.visualization
import ra_utils.visualization.plot_landmarks

## Setup imports and variables

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Loading the dataset

### Short description of the data and dataset module
The [landmarker](https://github.com/predict-idlab/landmarker) package has several built-in
datasets in the `landmarker.datasets` module, as well as utility classes for building your own
datasets in the `landmarker.data` module. There are three types of datasets: 'LandmarkDataset',
'HeatmapDataset', and 'MaskDataset'. The 'LandmarkDataset' is a dataset of images with landmarks,
the 'HeatmapDataset' is a dataset of images with heatmaps, and the 'MaskDataset' is a dataset of
images with masks (i.e., binary segmentation masks indiciating the location of the landmarks). The 
'HeatmapDataset' and 'MaskDataset' both inherit from the 'LandmarkDataset' class, and thus also 
contain information about the landmarks. The 'MaskDataset' can be constructed from specified image 
and landmarks pairs, or from images and masks pairs, because often that is how the data is
distributed. The 'HeatmapDataset' can be constructed from images and landmarks pairs.

Images can be provided as a list of paths to stored images, or as a a numpy arary, torch tensor, 
list of numpy  arrays or list of torch tensors. Landmarks can be as numpy arrays or torch tensors.
These landmarks can be provided in three different shapes: (1) (N, D) where N is the number of
samples and D is the number of dimensions, (2) (N, C, D) where C is the number of landmark
classes, (3) (N, C, I, D) where I is the number of instances per landmark class, if less than I
instances are provided, the remaining instances are filled with NaNs.

For built-in datasets, the `landmarker.datasets` module provides a function for
downloading and loading the dataset, e.g. `get_cepha_landmark_datasets` for the ISBI 2015 
Cephalometric X-ray dataset. Most of these functions take the same arguments, namely `path_dir`,
some are dataset specific. The `path_dir` argument specifies the directory where the dataset is
downloaded to, or loaded from if it is already downloaded. For most datasets multiple functions
are provided for getting different types of datasets. For example, the ISBI 2015 Cephalometric
X-ray dataset has the following functions: `get_cepha_landmark_datasets` and 
`get_cepha_heatmap_datasets`.

### Download and load ISBI 2015 landmark dataset
The ISBI 2015 Cephalometric X-ray dataset is a dataset of 2D cephalometric X-ray images with 19
landmarks. The dataset is split into a training set of 150 images and two test sets, where test
set  1 contains 150 images and test set 2 contains 100 images. When loading the dataset, you can
also specify a transform function, which is applied to the images and landmarks of the training
set. Currently, we only support the `monai.transforms.ComposeD transform, which allows you to
compose multiple transforms. The `monai.transforms` module contains many useful transforms, such
as `RandomAffine` and `NormalizeIntensity`. The transforms must be dictionary transforms, i.e.,
they must return a dictionary with the keys 'image' and ('seg'), in the case of heatmap and mask
regression.

In [ ]:
### Data check
# Is are the landmarks positioned as I expect prior to resizing? 
# Check a single image: 


data_dir = "/home/clemens/data/public/landmark-datasets"
i=0
lm_sample =  pd.read_csv(
        data_dir + f"/ISBI2015/400_junior/{str(i+1).zfill(3)}.txt",
        sep=",",
        header=None,
    )[:19].to_numpy()

image_path = str(list(Path(data_dir + f"/ISBI2015/RawImage/").glob(f"*/{str(i+1).zfill(3)}.bmp"))[0])

image = LoadImage(ensure_channel_first=True)(image_path)
image = Transpose((0,2,1))(image)
ra_utils.visualization.plot_landmarks.plot_landmarks(image[0,...].numpy(), 
                                                     landmarks=lm_sample, 
                                                     figsize=(4,3));



In [ ]:
lm_sample

In [ ]:




from landmarker.transforms.images import UseOnlyFirstChannel
fn_keys = ('image',)
spatial_transformd = [RandAffined(fn_keys, prob=1,
                        rotate_range=(-np.pi/12, np.pi/12),
                        translate_range=(-10, 10),
                        scale_range=(-0.1, 0.1),
                        shear_range=(-0.1, 0.1)
                        )]

train_transformd = Compose([
                            UseOnlyFirstChannel(('image', )),
                            RandGaussianNoised(('image', ), prob=0.2, mean=0, std=0.1),  # Add gaussian noise
                            RandScaleIntensityd(('image', ), factors=0.25, prob=0.2),  # Add random intensity scaling
                            RandAdjustContrastd(('image', ), prob=0.2, gamma=(0.5,4.5)),  # Randomly adjust contrast
                            RandHistogramShiftd(('image', ), prob=0.2),  # Randomly shift histogram
                            ScaleIntensityd(('image', )),  # Scale intensity
                        ] + spatial_transformd)

inference_transformd = Compose([
    UseOnlyFirstChannel(('image', )),
    ScaleIntensityd(('image', )),
])

In [ ]:


data_dir = "/home/clemens/data/public/landmark-datasets"
ds_train, ds_test1, ds_test2 = get_tiny_cepha_landmark_datasets(
    data_dir, train_transform=train_transformd,
    inference_transform=inference_transformd,
    store_imgs = True, dim_img=(512, 512),
    junior = True, 
    N_train=2,
    N_test1=2,
    N_test2=2)



In [ ]:
X = ds_train[0]

image=X["image"].numpy()[0,...]
landmarks = X["landmark"].numpy()

fig, ax = plt.subplots(figsize=(4,3))
ax.imshow(image, cmap='gray')
ax.scatter(landmarks[:, 1], landmarks[:, 0], 
            s=20, color='red', marker='x')
ax.axis("off")
plt.show()


## Constructing a heatmap generator
The heatmap generator is a class that generates heatmaps from landmarks. It is used to generate
heatmaps from the landmarks of the training set, which are then used to train the model. The
`landmarker.heatmap_generator` module contains several heatmap generators, such as the
`GaussianHeatmapGenerator` and `LaplaceHeatmapGenerator` which generate a multivariate 
Gaussian and Laplace distribution respectively. The HeatmapGenerator subclasses take the following
arguments:
* `sigmas`: the standard deviation of the Gaussian distribution, or the scale of the Laplace. This 
could be a scalar, or a list of scalars, one for each landmark class. Additionally, it could be a
covariance matrix, or a list of covariance matrices, one for each landmark class.
* `gamma`: If provided, the heatmaps are scaled by `gamma` before being returned.
* `rotation`: If provided, the heatmaps are rotated by `rotation` before being returned.
* `heatmap_size`: The size of the returned heatmaps.
* learnable: If True, the `sigma` and `rotation` parameters are learnable parameters, and thus
will be optimized during training.
* `background`: A boolean indicating whether to add a background class to the heatmaps. If True,
the heatmaps will have an additional channel, which is 1 everywhere except at the location of the
landmarks, where it is 0. The background class is the first class, i.e., the first channel.
* `all_points`: A boolean indicating whether to add a channel with all the landmarks. If True, the
heatmaps will have an additional channel, which is 1 at the location of the landmarks, and 0.
everywhere else.
* `continuous`: A boolean indicating whether to use continuous or discrete landmarks.
* `device`: The device on which the heatmaps are generated.

The landmarks provide to the heatmap generator must be a torch.Tensor and can be in three different 
shapes: (1) (N, D) where N is the number of samples and D is the number of dimensions, (2) (N, C, D)
where C is the number of landmark classes, (3) (N, C, I, D) where I is the number of instances per
landmark class, if less than I instances are provided, the remaining instances are filled with NaNs.
The heatmap generator will return a torch.Tensor of shape (N, C, H, W), where H and W are the height
and width of the heatmaps respectively.

**Note that with 2D landmarks the y coordinates are the first dimension, and the x coordinates are the
second dimension.**

In [ ]:
from landmarker.heatmap.generator import GaussianHeatmapGenerator

heatmap_generator = GaussianHeatmapGenerator(
    nb_landmarks=19,
    sigmas=3,
    gamma=100,
    heatmap_size=(512, 512),
    learnable=True, # If True, the heatmap generator will be trainable
)

## Inspecting the dataset

In [ ]:
from landmarker.visualize import inspection_plot

# Plot the first 3 images from the training set
inspection_plot(ds_train, range(2), heatmap_generator=heatmap_generator)

In [ ]:
# Plot the first 3 images from dataset without transforms
heatmap_generator.device = "cpu" # because dataset tensors are still on cpu
inspection_plot(ds_test1, 0, heatmap_generator=heatmap_generator)
heatmap_generator.device = device # set the desired device back

## Training and initializing the SpatialConfiguration model

### Initializing the model, optimizer and loss function

In [ ]:
from landmarker.models.spatial_configuration_net import OriginalSpatialConfigurationNet
from landmarker.losses import GaussianHeatmapL2Loss

model = OriginalSpatialConfigurationNet(in_channels=1, out_channels=19).to(device)
print("Number of learnable parameters: {}".format(
    sum(p.numel() for p in model.parameters() if p.requires_grad)))
lr = 1e-6
batch_size = 1
epochs = 2

optimizer = torch.optim.SGD([
    {'params': model.parameters(), "weight_decay":1e-3},
    {'params': heatmap_generator.sigmas},
    {'params': heatmap_generator.rotation}]
    , lr=lr, momentum=0.99, nesterov=True)


criterion = GaussianHeatmapL2Loss(
    alpha=5
)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,
                                                          patience=10, verbose=True, cooldown=10)

### Setting the data loaders

In [ ]:
train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(ds_test1, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(ds_test2, batch_size=batch_size, shuffle=False, num_workers=0)

### Training the model

In [ ]:
from landmarker.heatmap.decoder import heatmap_to_coord
from landmarker.metrics import point_error

def train_epoch(model, heatmap_generator, train_loader, criterion, optimizer, device):
    running_loss = 0
    model.train()
    for i, batch in enumerate(tqdm(train_loader)):
        images = batch["image"].to(device)
        landmarks = batch["landmark"].to(device)
        optimizer.zero_grad()
        outputs = model(images)
        heatmaps = heatmap_generator(landmarks)
        loss = criterion(outputs, heatmap_generator.sigmas, heatmaps)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

def val_epoch(model, heatmap_generator, val_loader, criterion, device, method="local_soft_argmax"):
    eval_loss = 0
    eval_mpe = 0
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            images = batch["image"].to(device)
            landmarks = batch["landmark"].to(device)
            outputs = model(images)
            dim_orig = batch["dim_original"].to(device)
            pixel_spacing = batch["spacing"].to(device)
            padding = batch["padding"].to(device)
            heatmaps = heatmap_generator(landmarks)
            loss = criterion(outputs, heatmap_generator.sigmas, heatmaps)
            pred_landmarks = heatmap_to_coord(outputs, method=method)
            eval_loss += loss.item()
            eval_mpe += point_error(landmarks, pred_landmarks, images.shape[-2:], dim_orig,
                                    pixel_spacing, padding, reduction="mean")
    return eval_loss / len(val_loader), eval_mpe / len(val_loader)

def train(model, heatmap_generator, train_loader, val_loader, criterion, optimizer, device, epochs=1000):
    for epoch in tqdm(range(epochs)):
        train_loss = train_epoch(model, heatmap_generator, train_loader, criterion, optimizer, device)
        val_loss, val_mpe = val_epoch(model, heatmap_generator, val_loader, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f} - Val mpe: {val_mpe:.4f}")
        lr_scheduler.step(val_loss)

In [ ]:
train(model, heatmap_generator, train_loader, val_loader, criterion, optimizer, device,
      epochs=epochs)

## Evaluating the model

In [ ]:
pred_landmarks = []
true_landmarks = []
dim_origs = []
pixel_spacings = []
paddings = []
test_mpe = 0
model.eval()
with torch.no_grad():
    for i, X in enumerate(tqdm(test_loader)):
        images = X["image"]
        landmarks = X["landmark"]
        affine_matrix = X["affine"]
        dim_orig = X["dim_original"] 
        pixel_spacing = X["spacing"]
        padding = X["padding"]
        
        images = images.to(device)
        landmarks = landmarks.to(device)
        dim_orig = dim_orig.to(device)
        pixel_spacing = pixel_spacing.to(device)
        padding = padding.to(device)
        outputs = model(images)
        # heatmap = heatmap_generator(landmarks)
        offset_coords = outputs.shape[1]-landmarks.shape[1]
        pred_landmark = heatmap_to_coord(outputs, offset_coords=offset_coords,
                                        method="local_soft_argmax")
        test_mpe += point_error(landmarks, pred_landmark, images.shape[-2:], dim_orig,
                                pixel_spacing, padding, reduction="mean")
        pred_landmarks.append(pred_landmark.cpu())
        true_landmarks.append(landmarks.cpu())
        dim_origs.append(dim_orig.cpu())
        pixel_spacings.append(pixel_spacing.cpu())
        paddings.append(padding.cpu())

pred_landmarks = torch.cat(pred_landmarks)
true_landmarks = torch.cat(true_landmarks)
dim_origs = torch.cat(dim_origs)
pixel_spacings = torch.cat(pixel_spacings)
paddings = torch.cat(paddings)

test_mpe /= len(test_loader)

print(f"Test Mean PE: {test_mpe:.4f}")

In [ ]:
from landmarker.metrics import sdr

sdr_test = sdr([2.0, 2.5, 3.0, 4.0], true_landmarks=true_landmarks, pred_landmarks=pred_landmarks,
               dim=(512, 512), dim_orig=dim_origs.int(), pixel_spacing=pixel_spacings, padding=paddings)
for key in sdr_test:
    print(f"SDR for {key}mm: {sdr_test[key]:.4f}")

In [ ]:
from landmarker.metrics import sdr

sdr_test = sdr([2.0, 2.5, 3.0, 4.0], true_landmarks=true_landmarks, pred_landmarks=pred_landmarks,
               dim=(512, 512), dim_orig=dim_origs.int(), pixel_spacing=pixel_spacings, padding=paddings)
for key in sdr_test:
    print(f"SDR for {key}mm: {sdr_test[key]:.4f}")

In [ ]:
from landmarker.visualize.utils import prediction_inspect_plot

model.eval()
model.to("cpu")
prediction_inspect_plot(ds_test2, model, range(2))

In [ ]:
from landmarker.visualize import detection_report

detection_report(true_landmarks, pred_landmarks, dim=(512, 512), dim_orig=dim_origs.int(),
                    pixel_spacing=pixel_spacings, padding=paddings, class_names=ds_train.class_names,
                    radius=[2.0, 2.5, 3.0, 4.0], digits=2)

In [ ]:
from landmarker.visualize import plot_cpe

plot_cpe(true_landmarks, pred_landmarks, dim=(512, 512), dim_orig=dim_origs.int(),
                    pixel_spacing=pixel_spacings, padding=paddings, class_names=ds_train.class_names,
                    group=False, title="CPE curve", save_path=None,
                    stat='proportion', unit='mm', kind='ecdf')